# CSV 파일 매칭 검증
같은 카테고리의 product CSV와 review CSV를 비교하여 불일치 확인

In [8]:
import pandas as pd
import glob
import os
import re

data_dir = '/Users/yu_seok/Documents/workspace/nbCamp/Project/Why-pi/data/csv'
product_files = glob.glob(os.path.join(data_dir, 'product_*.csv'))
review_files = glob.glob(os.path.join(data_dir, 'reviews_*.csv'))

print(f"product CSV: {len(product_files)}개")
print(f"reviews CSV: {len(review_files)}개\n")

def extract_category(filename):
    """product_메이크업_all_20260204.csv -> 메이크업_all_20260204"""
    basename = os.path.basename(filename)
    if basename.startswith('product_'):
        return basename.replace('product_', '').replace('.csv', '')
    elif basename.startswith('reviews_'):
        return basename.replace('reviews_', '').replace('.csv', '')
    return None

# 파일 매칭
product_dict = {extract_category(f): f for f in product_files}
review_dict = {extract_category(f): f for f in review_files}

print("="*80)
print("파일 매칭")
print("="*80)

all_categories = set(product_dict.keys()) | set(review_dict.keys())

for category in sorted(all_categories):
    has_product = category in product_dict
    has_review = category in review_dict
    
    status = "[OK]" if (has_product and has_review) else "[WARNING]"
    
    print(f"{status} {category}")
    if has_product:
        print(f"   product: {os.path.basename(product_dict[category])}")
    else:
        print(f"   product: 없음")
    
    if has_review:
        print(f"   reviews: {os.path.basename(review_dict[category])}")
    else:
        print(f"   reviews: 없음")
    print()

product CSV: 4개
reviews CSV: 4개

파일 매칭
[OK] 메이크업_베이스:립메이크업_20260204
   product: product_메이크업_베이스:립메이크업_20260204.csv
   reviews: reviews_메이크업_베이스:립메이크업_20260204.csv

[OK] 맨케어_all_20260204
   product: product_맨케어_all_20260204.csv
   reviews: reviews_맨케어_all_20260204.csv

[OK] 메이크업_아이메이크업_20260204
   product: product_메이크업_아이메이크업_20260204.csv
   reviews: reviews_메이크업_아이메이크업_20260204.csv

[OK] 메이크업_치크_하이라이터_20260204
   product: product_메이크업_치크_하이라이터_20260204.csv
   reviews: reviews_메이크업_치크_하이라이터_20260204.csv



In [9]:
matched_categories = set(product_dict.keys()) & set(review_dict.keys())

print("="*80)
print("카테고리별 product_code 매칭 검증")
print("="*80)
print()

total_only_product = 0
total_only_review = 0
total_matched = 0

for category in sorted(matched_categories):
    product_file = product_dict[category]
    review_file = review_dict[category]
    
    # CSV 읽기
    product_df = pd.read_csv(product_file)
    review_df = pd.read_csv(review_file)
    
    # product_code 추출
    product_codes = set(product_df['product_code'].dropna().astype(int))
    review_codes = set(review_df['product_code'].dropna().astype(int))
    
    # 매칭 확인
    both = product_codes & review_codes
    only_product = product_codes - review_codes
    only_review = review_codes - product_codes
    
    total_matched += len(both)
    total_only_product += len(only_product)
    total_only_review += len(only_review)
    
    print(f"{'='*80}")
    print(f"[{category}]")
    print(f"{'='*80}")
    print(f"  product CSV: {len(product_codes)}개")
    print(f"  reviews CSV: {len(review_codes)}개")
    print()
    print(f"  [OK] 매칭됨 (product & review 모두): {len(both)}개")
    
    if len(only_product) > 0:
        print(f"  [WARNING] product만 있음 (리뷰 없음): {len(only_product)}개")
        if len(only_product) <= 10:
            print(f"     코드: {sorted(only_product)}")
        else:
            print(f"     처음 10개: {sorted(only_product)[:10]}")
    
    if len(only_review) > 0:
        print(f"  [ERROR] review만 있음 (제품 정보 없음): {len(only_review)}개")
        if len(only_review) <= 10:
            print(f"     코드: {sorted(only_review)}")
        else:
            print(f"     처음 10개: {sorted(only_review)[:10]}")
    
    print()

카테고리별 product_code 매칭 검증

[메이크업_베이스:립메이크업_20260204]
  product CSV: 237개
  reviews CSV: 237개

  [OK] 매칭됨 (product & review 모두): 237개

[맨케어_all_20260204]
  product CSV: 73개
  reviews CSV: 72개

  [OK] 매칭됨 (product & review 모두): 72개
  [WARNING] product만 있음 (리뷰 없음): 1개
     코드: [1059931]

[메이크업_아이메이크업_20260204]
  product CSV: 128개
  reviews CSV: 124개

  [OK] 매칭됨 (product & review 모두): 124개
  [WARNING] product만 있음 (리뷰 없음): 4개
     코드: [1069087, 1069088, 1069089, 1069090]

[메이크업_치크_하이라이터_20260204]
  product CSV: 81개
  reviews CSV: 81개

  [OK] 매칭됨 (product & review 모두): 81개



In [10]:
# 전체 요약
print(f"\n{'='*80}")
print("전체 요약")
print(f"{'='*80}")
print(f"  검증한 카테고리: {len(matched_categories)}개")
print(f"  [OK] 매칭됨: {total_matched}개")
print(f"  [WARNING] product만 있음: {total_only_product}개")
print(f"  [ERROR] review만 있음: {total_only_review}개")
print()

if total_only_product == 0 and total_only_review == 0:
    print("  [OK] 모든 파일이 완벽하게 매칭됩니다!")
elif total_only_review > 0:
    print(f"  [ERROR] {total_only_review}개 제품의 리뷰만 있고 제품 정보가 없습니다!")
elif total_only_product > 0:
    print(f"  [INFO] {total_only_product}개 제품은 리뷰가 없습니다 (정상)")


전체 요약
  검증한 카테고리: 4개
  [OK] 매칭됨: 514개
  [WARNING] product만 있음: 5개
  [ERROR] review만 있음: 0개

  [INFO] 5개 제품은 리뷰가 없습니다 (정상)


In [11]:
all_products = []

for product_file in product_files:
    df = pd.read_csv(product_file)
    if 'brand' in df.columns and 'name' in df.columns:
        all_products.append(df[['brand', 'name', 'product_code']])

# 전체 데이터 병합
if all_products:
    combined_df = pd.concat(all_products, ignore_index=True)
    
    print("="*80)
    print("전체 통계")
    print("="*80)
    print(f"총 제품 수: {len(combined_df)}개")
    print(f"고유 브랜드 수: {combined_df['brand'].nunique()}개")
    print()
    
    # 브랜드별 제품 수
    brand_counts = combined_df['brand'].value_counts()
    
    print("="*80)
    print("브랜드별 제품 수")
    print("="*80)
    for brand, count in brand_counts.items():
        if pd.notna(brand) and brand != '':
            print(f"{brand}: {count}개")
        else:
            print(f"[브랜드 없음]: {count}개")
    
    print()
    print("="*80)
    print("브랜드별 상품 목록")
    print("="*80)
    
    # 브랜드별로 그룹핑하여 상품명 출력
    for brand in sorted(combined_df['brand'].unique()):
        if pd.isna(brand) or brand == '':
            brand_name = "[브랜드 없음]"
        else:
            brand_name = brand
        
        products = combined_df[combined_df['brand'] == brand]
        
        print(f"\n[{brand_name}] ({len(products)}개)")
        print("-"*80)
        
        for idx, row in products.iterrows():
            print(f"  - {row['name']}")
else:
    print("제품 데이터가 없습니다.")

전체 통계
총 제품 수: 519개
고유 브랜드 수: 71개

브랜드별 제품 수
태그: 42개
머지: 36개
투에딧: 32개
프릴루드 딘토: 31개
입큰: 30개
본셉 메이크업: 29개
트윙클팝: 27개
손앤박: 26개
플레이101 by 에뛰드: 25개
밀크터치 디어씽: 21개
드롭비 컬러즈: 19개
줌 바이 정샘물: 11개
코시에로: 10개
코코가가: 10개
클라뷰: 9개
스타일리쉬: 8개
더봄: 7개
베리썸: 7개
쉬크: 7개
[01: 7개
[02: 7개
싸이닉: 6개
프렙 바이 비레디: 6개
더랩 바이 블랑두: 6개
데일리콤마: 6개
어퓨_더퓨어: 5개
도루코: 5개
보닌: 4개
KAI: 4개
NEW: 4개
파넬: 4개
애교살: 3개
윙크걸: 3개
러블리: 3개
코드글로컬러: 3개
[03: 3개
오릭스: 3개
하드: 2개
맨넨: 2개
닥터지오: 2개
[04: 2개
도루코윈3: 2개
도루코윈4: 2개
6중날: 2개
도루코윈: 2개
멘넨: 2개
제이엠솔루션: 2개
미팩토리: 2개
클리덤: 2개
식물원: 2개
플랫형: 2개
펜슬: 2개
펠트: 2개
스니키: 1개
눈썹: 1개
3중: 1개
둥근형: 1개
도루코TG2플러스5P: 1개
3중날: 1개
2중날: 1개
펜슬＆브러쉬: 1개
방수: 1개
남성용다리털정리기: 1개
그로우어스: 1개
전동: 1개
심플: 1개
비알티씨: 1개
캐릭터: 1개
수채화: 1개
잇츠스킨: 1개
카이지루시: 1개

브랜드별 상품 목록

[2중날] (1개)
--------------------------------------------------------------------------------
  - 2중날 휴대용 면도기 10개입

[3중] (1개)
--------------------------------------------------------------------------------
  - 3중 날 시스템 면도기 리필 4개입

[3중날] (1개)
-----------------------------------------------

# Parquet 파일 생성
모든 CSV 파일을 하나의 products.parquet과 reviews.parquet으로 통합

In [15]:
data_dir = '/Users/yu_seok/Documents/workspace/nbCamp/Project/Why-pi/data/csv'

# 1. Product CSV 파일들 통합
print("="*80)
print("Product CSV 파일 통합")
print("="*80)

product_files = glob.glob(os.path.join(data_dir, 'product_*.csv'))
print(f"발견된 product CSV: {len(product_files)}개\n")

all_products = []
for product_file in product_files:
    df = pd.read_csv(product_file)
    print(f"  - {os.path.basename(product_file)}: {len(df)}개 제품")
    all_products.append(df)

# Product 데이터 통합
if all_products:
    products_df = pd.concat(all_products, ignore_index=True)
    
    # 중복 제거 (product_code 기준)
    before_dedup = len(products_df)
    products_df = products_df.drop_duplicates(subset=['product_code'], keep='first')
    after_dedup = len(products_df)
    
    print(f"\n통합 전 총 제품 수: {before_dedup}개")
    print(f"중복 제거 후: {after_dedup}개 (중복 {before_dedup - after_dedup}개 제거)")
    
    # Parquet으로 저장
    output_path = os.path.join(data_dir, 'products.parquet')
    products_df.to_parquet(output_path, index=False, engine='pyarrow')
    print(f"\n✓ 저장 완료: {output_path}")
    print(f"  파일 크기: {os.path.getsize(output_path):,} bytes")
else:
    print("Product 데이터가 없습니다.")

print()

Product CSV 파일 통합
발견된 product CSV: 4개

  - product_메이크업_치크_하이라이터_20260204.csv: 81개 제품
  - product_메이크업_아이메이크업_20260204.csv: 128개 제품
  - product_메이크업_베이스:립메이크업_20260204.csv: 237개 제품
  - product_맨케어_all_20260204.csv: 73개 제품

통합 전 총 제품 수: 519개
중복 제거 후: 519개 (중복 0개 제거)

✓ 저장 완료: /Users/yu_seok/Documents/workspace/nbCamp/Project/Why-pi/data/csv/products.parquet
  파일 크기: 34,021 bytes



In [16]:
# 2. Reviews CSV 파일들 통합
print("="*80)
print("Reviews CSV 파일 통합")
print("="*80)

review_files = glob.glob(os.path.join(data_dir, 'reviews_*.csv'))
print(f"발견된 reviews CSV: {len(review_files)}개\n")

all_reviews = []
for review_file in review_files:
    df = pd.read_csv(review_file)
    print(f"  - {os.path.basename(review_file)}: {len(df)}개 리뷰")
    all_reviews.append(df)

# Reviews 데이터 통합
if all_reviews:
    reviews_df = pd.concat(all_reviews, ignore_index=True)
    
    # 중복 제거 (user, product_code, date, text 조합으로 판단)
    before_dedup = len(reviews_df)
    reviews_df = reviews_df.drop_duplicates(
        subset=['user', 'product_code', 'date', 'text'], 
        keep='first'
    )
    after_dedup = len(reviews_df)
    
    print(f"\n통합 전 총 리뷰 수: {before_dedup}개")
    print(f"중복 제거 후: {after_dedup}개 (중복 {before_dedup - after_dedup}개 제거)")
    
    # Parquet으로 저장
    output_path = os.path.join(data_dir, 'reviews.parquet')
    reviews_df.to_parquet(output_path, index=False, engine='pyarrow')
    print(f"\n✓ 저장 완료: {output_path}")
    print(f"  파일 크기: {os.path.getsize(output_path):,} bytes")
    
    # 통계 출력
    print(f"\n리뷰 통계:")
    print(f"  고유 제품 수: {reviews_df['product_code'].nunique()}개")
    print(f"  고유 사용자 수: {reviews_df['user'].nunique()}명")
else:
    print("Reviews 데이터가 없습니다.")

print()

Reviews CSV 파일 통합
발견된 reviews CSV: 4개

  - reviews_메이크업_치크_하이라이터_20260204.csv: 12883개 리뷰
  - reviews_메이크업_아이메이크업_20260204.csv: 24386개 리뷰
  - reviews_메이크업_베이스:립메이크업_20260204.csv: 55710개 리뷰
  - reviews_맨케어_all_20260204.csv: 5651개 리뷰

통합 전 총 리뷰 수: 98630개
중복 제거 후: 98379개 (중복 251개 제거)

✓ 저장 완료: /Users/yu_seok/Documents/workspace/nbCamp/Project/Why-pi/data/csv/reviews.parquet
  파일 크기: 5,968,328 bytes

리뷰 통계:
  고유 제품 수: 514개
  고유 사용자 수: 12335명



In [17]:
# 찐최종 검증
print("="*80)
print("최종 검증")
print("="*80)

# Parquet 파일 다시 읽기
products_path = os.path.join(data_dir, 'products.parquet')
reviews_path = os.path.join(data_dir, 'reviews.parquet')

if os.path.exists(products_path) and os.path.exists(reviews_path):
    products_check = pd.read_parquet(products_path)
    reviews_check = pd.read_parquet(reviews_path)
    
    print(f"\nproducts.parquet:")
    print(f"  레코드 수: {len(products_check):,}개")
    print(f"  컬럼 수: {len(products_check.columns)}개")
    print(f"  컬럼: {', '.join(products_check.columns.tolist())}")
    
    print(f"\nreviews.parquet:")
    print(f"  레코드 수: {len(reviews_check):,}개")
    print(f"  컬럼 수: {len(reviews_check.columns)}개")
    print(f"  컬럼: {', '.join(reviews_check.columns.tolist())}")
    
    # 매칭 검증
    product_codes_in_products = set(products_check['product_code'].unique())
    product_codes_in_reviews = set(reviews_check['product_code'].unique())
    
    matched = product_codes_in_products & product_codes_in_reviews
    only_in_products = product_codes_in_products - product_codes_in_reviews
    only_in_reviews = product_codes_in_reviews - product_codes_in_products
    
    print(f"\n{'='*80}")
    print("매칭 검증")
    print(f"{'='*80}")
    print(f"  [OK] 양쪽에 모두 있는 제품: {len(matched)}개")
    print(f"  [INFO] product만 있음 (리뷰 없음): {len(only_in_products)}개")
    print(f"  [WARNING] review만 있음 (제품 정보 없음): {len(only_in_reviews)}개")
    
    # 매칭 안되는 제품 상세 정보
    if len(only_in_products) > 0:
        print(f"\n{'='*80}")
        print("리뷰가 없는 제품 (product만 있음)")
        print(f"{'='*80}")
        no_review_products = products_check[products_check['product_code'].isin(only_in_products)]
        for idx, row in no_review_products.iterrows():
            print(f"\n  제품코드: {row['product_code']}")
            print(f"  브랜드: {row['brand']}")
            print(f"  제품명: {row['name']}")
            print(f"  카테고리: {row['category_1']} > {row['category_2']}")
            print(f"  가격: {row['price']}원")
    
    if len(only_in_reviews) > 0:
        print(f"\n{'='*80}")
        print("제품 정보가 없는 리뷰 (review만 있음)")
        print(f"{'='*80}")
        print(f"  제품 코드: {sorted(only_in_reviews)}")
        
        # 리뷰만 있는 제품의 리뷰 수 확인
        orphan_reviews = reviews_check[reviews_check['product_code'].isin(only_in_reviews)]
        review_counts = orphan_reviews['product_code'].value_counts()
        print(f"\n  제품별 리뷰 수:")
        for code, count in review_counts.items():
            print(f"    제품코드 {code}: {count}개 리뷰")
    
    print(f"\n{'='*80}")
    print("✓ Parquet 파일 생성 및 검증 완료!")
    print(f"{'='*80}")
else:
    print("Parquet 파일이 생성되지 않았습니다.")

최종 검증

products.parquet:
  레코드 수: 519개
  컬럼 수: 14개
  컬럼: product_code, category_home, category_1, category_2, brand, name, price, country, likes, shares, url, can_할랄인증, can_비건, certifications

reviews.parquet:
  레코드 수: 98,379개
  컬럼 수: 7개
  컬럼: product_code, date, user_masked, user, rating, text, image_count

매칭 검증
  [OK] 양쪽에 모두 있는 제품: 514개
  [INFO] product만 있음 (리뷰 없음): 5개
  [WARNING] review만 있음 (제품 정보 없음): 0개

리뷰가 없는 제품 (product만 있음)

  제품코드: 1069087
  브랜드: [01
  제품명: [01 딥브라운] 더봄 스키니브로우 펜슬
  카테고리: 메이크업 > 아이메이크업
  가격: 2000원

  제품코드: 1069088
  브랜드: [02
  제품명: [02 내추럴브라운] 더봄 스키니브로우 펜슬
  카테고리: 메이크업 > 아이메이크업
  가격: 2000원

  제품코드: 1069090
  브랜드: [04
  제품명: [04 토프베이지] 더봄 스키니브로우 펜슬
  카테고리: 메이크업 > 아이메이크업
  가격: 2000원

  제품코드: 1069089
  브랜드: [03
  제품명: [03 소프트그레이] 더봄 스키니브로우 펜슬
  카테고리: 메이크업 > 아이메이크업
  가격: 2000원

  제품코드: 1059931
  브랜드: KAI
  제품명: KAI 2중날 고정식 면도기 5개입
  카테고리: 맨케어 > 남성용면도기
  가격: 2000원

✓ Parquet 파일 생성 및 검증 완료!
